## 2nd Model : 2 Hidden Layers
- Network Architecture: 
    - 2 hidden layers
    - ReLU Activation on Hidden
    - Sigmoid Activation on Output
    - learning rate = 0.01
    - epoch = 500

## Steps:
1. Import Libraries 
2. Read Data from `\Dataset` folder (Retrieved from UCI Repositories)
3. Clean Dataset (rename and format the column)
4. Data Preprocessing
5. Initialize the model
6. Train and test the data
7. Evaluate Performance using graph

### 0. Install Library Requirements

In [ ]:
# %pip install jupyter numpy pandas matplotlib seaborn tabulate imblearn

### 1. Import Libraries
Import necessary libraries:
* `numpy`     : High performance array object
* `pandas`    : Data analysis and manipulation tools
* `matplotlib`: Data visualizations
* `seaborn`   : Data visualizations framework based on **matplotlib**
* `tabulate`  : Pretty-print tabular structure library

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tabulate import tabulate
import seaborn as sns

#### 1.1 Helper Functions
* Handle mathematical operation used in the `NeuralNetwork` Class

In [ ]:
def sigmoid(x): return 1 /(1 + np.exp(-x))
def relu(x): return np.maximum(0,x)
def derivative_relu(relu_x): return (relu_x>0).astype(float)
def binary_cross_entropy(y_true, y_pred, epsilon=1e-15):
    # Clip predictions to prevent log(0)
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

### 2. Read and Display Dataset Information
load the dataset and understand the data, and use the Interquartile Range (IQR) method to identify potential outliers in numerical columns.
2.1 Read dataset

In [ ]:
def read_file(file_path: str):
    df = pd.read_csv(file_path)
    new_columns = [col.strip().replace('  ', ' ').replace(' ', '_').lower() for col in df.columns]
    df.columns = new_columns
    return df

path = r'..\Dataset\Customer Churn.csv'
df = read_file(path)
df.head(20)

2.2 Display dataset data types and null values

In [ ]:
df.info()

2.3 Display dataset shape and its statisical values

In [ ]:
df.describe()

2.4. Detect Outliers

In [ ]:
def detect_outliers_iqr(df, k=1.5):
    nums = df.select_dtypes(include='number')
    outlier_info = {}
    
    for c in nums.columns:
        #skip binary columns
        if(nums[c].nunique() <= 2):
            continue
        
        q1 = nums[c].quantile(0.25) #1st quartile
        q3 = nums[c].quantile(0.75) #3rd quartile
        iqr = q3 - q1
        lower = q1 - k * iqr
        upper = q3 + k * iqr
        mask = (nums[c] < lower) | (nums[c] > upper)
        outlier_info[c] = {
            'count': int(mask.sum()),
            'indices': nums.index[mask].tolist(),
            'lower': float(lower),
            'upper': float(upper)
        }
    return outlier_info

outliers = detect_outliers_iqr(df)
table_data = []
print('\nOutlier summary (IQR method):')
for col, info in outliers.items():
    if info['count'] > 0:
    # Calculate percentage
        perc = (info["count"] / len(df)) * 100
        
        # Add a list (row) to our table_data
        table_data.append([col, info["count"], f"{info['lower']:.3f}",
            f"{info['upper']:.3f}", f"{perc:.2f}%"])
headers = ["Column", "Outlier Count", "Lower Bound", "Upper Bound", "Percentage"]
print(tabulate(table_data, headers=headers))
    

### 3. Data Cleaning 
This process take step before handling with outliers and data preprocessing.

3.1 Remove Duplicate Row (if any)

In [ ]:
df = df.drop_duplicates()
print(f'\n[Changes] Removed duplicate rows. New shape={df.shape}\n')

3.2 Remove Redundant Groups

`age_group` and `age` columns both have the same values but in different types, numeric and nominal respectively.
`age_group` column is dropped to prevent biases when learning

In [ ]:
df = df.drop(columns=['age_group'])
print(f'\n[Changes] Dropped column: age_group due to redundancy. New shape={df.shape}\n\n')


### 4. Data Preprocessing

The dataset will follows the exact steps to avoid data leakage during testing.

[Split] -> [Log] -> [Encode] -> [Fit] -> [Scale]

#### 4.1 Split

* Splitting the dataset into train, and test sets, and both input,X and output,Y
* This function will reset the intialized index of X and Y dataset, and shuffle them before splitting to prevent **data leakage**

@function `split_data()`:
* params
    * `X`, `y` as input and output
    * `test_split` as testing split percentage ; default `0.2`
    * `randomness` as random values for randomize dataset index before splitting ; default `None`
* return
    * `X_train` = Training set without `Churn`
    * `Y_train` = `Churn` Training column
    * `Y_train` = Testing set without `Churn`
    * `Y_test` = `Churn` Testing column


In [ ]:
def split_data(X, y, test_split=0.2, randomness=None):
    # Set seed for reproducibility
    if randomness is not None:
        np.random.seed(randomness)
    
    # reset X and Y current index
    X = X.reset_index(drop=True)
    y = y.reset_index(drop=True)
    
    # Identify unique classes and their indices (0 and 1)
    unique_classes = np.unique(y)
    train_indices = []
    test_indices = []
    
    for cls in unique_classes:
        # Get indices of rows belonging to this class
        cls_indices = np.where(y == cls)[0]

        # Shuffle indices within this specific class
        np.random.shuffle(cls_indices)

        # Determine the split point
        total_count = len(cls_indices)
        test_count = int(total_count * test_split)
        
        # Split indices
        cls_test = cls_indices[:test_count]
        cls_train = cls_indices[test_count:]
        
        # Add to main lists
        test_indices.extend(cls_test)
        train_indices.extend(cls_train)
        
    # Shuffle the final combined indices so they aren't grouped by class
    np.random.shuffle(train_indices)
    np.random.shuffle(test_indices)
    
    # Use .iloc for DataFrames to select the rows
    X_train, X_test = X.iloc[train_indices], X.iloc[test_indices]
    y_train, y_test = y.iloc[train_indices], y.iloc[test_indices]
    return X_train, X_test, y_train, y_test

X = df.drop(columns=['churn'], axis=1)
Y = df['churn']
X_train,X_test,y_train,y_test = split_data(X,Y,test_split=0.2, randomness=42)
print(f'[Changes] Successfully split data into Training and Testing.')


#### 4.2 Log Transformation

* This will treat the outliers found before, compressing all the values within the dataset to **reduce** the skewness of the data
* This step is to prevent unfair patterns.
* Use `numpy.log(1 + x)`
* log transformation on seperated train and test set to prevent **data leakage**

@function `log_transformation()`:
* params
    * `df_train`, `df_test` as training and testing dataset
    * `cols_log` as list of column to be log transformed
* return
    * `df_train` = train dataset that have been log transformed
    * `df_test` = test dataset that have been log transformed

In [ ]:
def log_transformation(df_train, df_test, cols_log: list):
    for col in cols_log:
        df_train[col] = np.log1p(df_train[col])
        df_test[col] = np.log1p(df_test[col])
    print(f'[Changes] Applied log transformation to selected columns.')
    return df_train, df_test

cols_to_log = [
    'seconds_of_use',
    'frequency_of_use',
    'frequency_of_sms',
    'distinct_called_numbers',
    'call_failure',
    'customer_value',
    'charge_amount'
]
X_train, X_test = log_transformation(X_train,X_test,cols_to_log)
    

#### 4.3 One-Hot Encoding

* Convert categorical column into multiple, numeric dummy columns that values between `0` and `1`.
* Example: `plan` have 3 categories  = `[cat1, cat2, cat3]`. It will split into dummy columns for each category, `plan_cat1`, `plan_cat2`, `plan_cat3` 
* use drop first column attribute to prevent **multicollinearity**

@function `get_train_category()`:
* params
    * `df` as dataset
    * `col_name` retrieve category from this column
* return
    * `list` = list of column name


@function `one_hot_encoding()`:
* params
    * `df` as dataset
    * `col_name` as column name that will apply one-hot encoding
    * `categories` as category inside `col_name`
    * `drop_first` drop first column to simplify the dummy column
* return
    * `converted_pd` = return the dataframe with the added dummy columns 

In [ ]:
def get_train_categories(df, col_name):
    return sorted(list(set(df[col_name].tolist())))

def one_hot_encoding(df, col_name:str, categories, drop_first = True):
    data = df[col_name].tolist()
    
    active_cats = categories[1:] if drop_first and len(categories) > 1 else categories
    
    encoded_mtx = []
    for item in data:
        row = [0] * len(active_cats)
        if item in active_cats:
            index = active_cats.index(item)
            row[index] = 1
        encoded_mtx.append(row)
    
    #rename column for one hot encoded column
    new_cols = [f'{col_name}_{cat}' for cat in active_cats]
    #convert back to dataframe
    converted_pd  = pd.DataFrame(encoded_mtx, columns=new_cols,index=df.index)
    return converted_pd

train_encoded_parts = []
test_encoded_parts = []

categorical = ['complains', 'tariff_plan', 'status']
for col in categorical:
    train_categories = get_train_categories(X_train, col)
    train_encoded_parts.append(one_hot_encoding(X_train, col, train_categories, drop_first=True))        
    test_encoded_parts.append(one_hot_encoding(X_test, col, train_categories, drop_first=True))
print(f'[Changes] Applied one hot encoding to categorical columns')

#drop old column and join new columns
X_train = X_train.drop(columns=categorical).join(train_encoded_parts)
X_test = X_test.drop(columns=categorical).join(test_encoded_parts)

#### 4.4 Scaling Feature

* Transforms features to fit within a specific range, usually `0`, by subtracting the minimum value and dividing by the range `(max - min)`
* Using `min()` and `max()` function to retrieve the minimum and maximum value in a column
* Scales the transformation on training data, and transform on train and test dataset to prevent data leakage

@function `get_scaling_params()`:
* params
    * `df_train` as training input dataset
* return
    * `param_dict` = a dictionary contains the minimum and maximum value for every key column

@function `min_max_transform()`:
* params
    * `df` as input dataset
    * `params` as dicitionary containing min and max value for each column
* return
    * `df_scaled` = scaled dataset 

In [ ]:
def get_scaling_params(df_train):
    param_dict = {}
    for col in df_train.columns:   
        min_val = min(df_train[col])
        max_val = max(df_train[col])
        param_dict[col] = (min_val, max_val - min_val)
    return param_dict

#eg: min_max_transform(X_TEST, train_min_val, train_data_range)
def min_max_transform(df, params: dict):
    df_scaled = df.copy()

    for col, (min_v, data_range) in params.items():
        if data_range == 0:
            df_scaled[col] = 0.0 #base case float number
        else:
            df_scaled[col] = (df[col] - min_v) / data_range
            
    return df_scaled

X_train_scale_params = get_scaling_params(X_train)
X_train_scaled = min_max_transform(X_train, X_train_scale_params)
X_test_scaled = min_max_transform(X_test, X_train_scale_params)
print(f'[Changes] Applied Min Max Scaler on numerical columns')

### 5. Initialize the Model

#### 5.1 Intialize Variables
* `EPOCHS`      : Iteration counts
* `PATIENCE`    : Hyperparameter stopping condition that wait for an improvement
* `input_dim` = input layer size (nodes)

In [ ]:
EPOCHS = 500
PATIENCE = 100

input_dim = X_train_scaled.shape[1]

@Class `NeuralNetwork`
* A multi-layer perceptron (Backpropagation Neural Network) designed for binary classification.
* He initialization for weight stability, gradient descent, and early stopping based on patience or error thresholds.

@function `feedforward()`:
* Feed the data forward, [Input] -> w1 -> [Hidden] -> w2 -> [Output]
* Input to hidden layer will use **ReLU** activation function to handle nonlinear
* Hidden layer to output layer will use **Sigmoid** activation function to **0 - 1** output

@function `backpropagation()`:
* Calculates the gradients of the loss function with respect to each weight and bias in the network.
* Uses Gradient Clipping (max value of 5.0) to prevent exploding gradients and stability.
* Updates weight1, weight2, bias1, and bias2 using the calculated deltas and the learning rate (`alpha`).

@function `train()`
* Manages the entire learning process over a set number of EPOCHS using  Gradient Descent.
* Shuffles the training data every epoch to improve generalization and prevent the model from learning the order of the data.
* Stopping Condition:
    * `max_error`
    * `patience`

@function `calculate_accuracy()`
* Computes the percentage of correct predictions by comparing predicted probabilities against true labels
* 0.5 threshold to convert sigmoid probabilities into binary classes (0 or 1).

@function `predict()`
* Performs a final feedforward pass on unseen data to generate class predictions.
* Flattens the final output and applies a boolean threshold (0.5) to return integer class labels.

In [ ]:
class NeuralNetwork:
    def __init__(self, input_dimension, hidden1_nodes=16, hidden2_nodes=8, alpha=0.01):
        #Step 0: Initialization
        self.alpha = alpha
        self.weight1 = np.random.randn(input_dimension, hidden1_nodes) * np.sqrt(2/input_dimension) #He Initializaion keeps Weight stable
        self.weight2 = np.random.randn(hidden1_nodes, hidden2_nodes) * np.sqrt(2/hidden1_nodes)
        self.weight3 = np.random.randn(hidden2_nodes, 1) * np.sqrt(2/hidden2_nodes)
        self.bias1 = np.zeros((1, hidden1_nodes))
        self.bias2 = np.zeros((1, hidden2_nodes))
        self.bias3 = np.zeros((1, 1))
        
        self.train_loss, self.test_loss, self.train_acc, self.test_acc = [], [], [], []
        self.history = {
            'train_loss': self.train_loss,
            'test_loss': self.test_loss,
            'train_acc': self.train_acc,
            'test_acc': self.test_acc
        }    
        
    def feedforward(self,X):
        # Step 1 : Calc Hidden Layer
        self.hidden1_Z = X @ self.weight1 + self.bias1
        self.hidden1_A = relu(self.hidden1_Z)
        self.hidden2_Z = self.hidden1_A @ self.weight2 + self.bias2
        self.hidden2_A = relu(self.hidden2_Z)
        
        # Step 2: Calc Output Layer
        self.output_Z = self.hidden2_A @ self.weight3 + self.bias3
        self.output_A= sigmoid(self.output_Z)
        
        return self.output_A
    
    def backpropagation(self, X, y, output):
        size = y.shape[0] 
        
        # Step 3: Calculate Error
        d_output = (output - y)
        
        # Step 4: Calculate Output Error Gradient
        d_weight3 = self.hidden2_A.T @ d_output / size
        d_bias3 = np.sum(d_output, axis=0, keepdims=True) / size
        
        # Step 5: Calculate Hidden Error Gradient
        d_hidden2_A = d_output @ self.weight3.T
        d_hidden2_Z = d_hidden2_A * derivative_relu(self.hidden2_Z)
        d_weight2 = self.hidden1_A.T @ d_hidden2_Z / size
        d_bias2 = np.sum(d_hidden2_Z, axis=0, keepdims=True) / size
        d_hidden1_A = d_hidden2_Z @ self.weight2.T
        d_hidden1_Z = d_hidden1_A * derivative_relu(self.hidden1_Z)
        d_weight1 = X.T @ d_hidden1_Z / size
        d_bias1 = np.sum(d_hidden1_Z, axis=0, keepdims=True) / size
        
        # Step 6: Update Output Weight
        self.weight3 -= self.alpha * d_weight3
        self.bias3 -= self.alpha * d_bias3
        
        # Step 7 and 8: Update Hidden Weight
        self.weight2 -= self.alpha * d_weight2
        self.bias2 -= self.alpha * d_bias2
        self.weight1 -= self.alpha * d_weight1
        self.bias1 -= self.alpha * d_bias1

    def calculate_accuracy(self, y_true, y_pred_prob):
        # Threshold at 0.5 for binary classification
        predictions = (y_pred_prob > 0.5).astype(int)
        correct = np.sum(predictions == y_true)
        return correct / len(y_true)
    
    def train(self, X_train, y_train, X_test, y_test):
        epochs = EPOCHS
        max_error = 0.01
        best_loss = float('inf')
        patience_count = 0
        patience = PATIENCE
        
        X_tr = X_train.values if isinstance(X_train, pd.DataFrame) else X_train
        y_tr = np.array(y_train).reshape(-1, 1)
        X_te = X_test.values if isinstance(X_test, pd.DataFrame) else X_test
        y_te = np.array(y_test).reshape(-1, 1)
        
        n_samples = X_tr.shape[0]
        
        for epoch in range(epochs):
            # Calculate metrics on full dataset
            output_full = self.feedforward(X_tr)
            self.backpropagation(X_tr, y_tr, output_full)
            
            train_loss = binary_cross_entropy(y_tr, output_full)
            train_acc = np.mean((output_full > 0.5).astype(int) == y_tr) * 100
            
            output_test = self.feedforward(X_te)
            test_loss = binary_cross_entropy(y_te, output_test)
            test_acc = np.mean((output_test > 0.5).astype(int) == y_te) * 100
            
            self.train_loss.append(train_loss)
            self.test_loss.append(test_loss)
            self.train_acc.append(train_acc)
            self.test_acc.append(test_acc)
            
            print(f'Epoch {epoch + 1}/{epochs} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}% | Test Loss: {test_loss:.4f} | Test Acc: {test_acc:.2f}%')
            
            if test_loss < best_loss:
                best_loss = test_loss
                patience_count = 0
            else:
                patience_count += 1
            
            if patience_count >= patience:
                print(f'[Training Stopped] Patience {patience} reached')
                break
            
            if train_loss <= max_error:
                print(f'[Training Stopped] Max error {max_error} reached')
                break
            
        return self.history

    def predict(self, X):
        X_vals = X.values if isinstance(X, pd.DataFrame) else X        
        h1_Z = X_vals @ self.weight1 + self.bias1
        h1_A = relu(h1_Z)
        h2_Z = h1_A @ self.weight2 + self.bias2
        h2_A = relu(h2_Z)
        o_Z = h2_A @ self.weight3 + self.bias3
        probs = sigmoid(o_Z)
        return (probs > 0.5).astype(int).flatten()

5.2 Initialize, Train and Predict

* Input Layer : 12 nodes
* Hidden Layer 1 : 16 nodes
* Hidden Layer 2 : 8 nodes
* Learning Rate : 0.01
* Epochs : 500

In [ ]:

nn = NeuralNetwork(input_dimension= input_dim, alpha=0.01)
history = nn.train(X_train_scaled, y_train, X_test_scaled, y_test)    
y_hat = nn.predict(X_test_scaled)

print('Model finished training and validation.')


@function `evaluate_model_performance()`:

* Generates a comprehensive visual report of the model's training history and final classification performance in a 3-row vertical layout.

* **Loss Curve**: Visualizes the relationship between training and validation loss across epochs to detect overfitting or convergence issues.

* **Accuracy Curve** : Tracks the percentage of correct predictions over time for both the training and test sets.

* **Confusion Matrix** : Calculate the classification metrics (TP, TN, FP, FN) and displays them as a Seaborn heatmap for error analysis.

* params
    * `history`: A dictionary containing the logged lists of loss and accuracy values.
    * `y_true` : The truth labels from the test dataset.
    * `y_pred` : The binary predictions (0 or 1) generated by the model.
    * `title` : string used as the main heading for the entire figure.

In [ ]:
def evaluate_model_performance(history:dict, y_true:list, y_pred:list, title:str):
    # Create a figure within 3 rows
    fig, axes = plt.subplots(3, 1, figsize=(6, 20))
    fig.suptitle(f'Model Evaluation: {title}', fontsize=16, fontweight='bold')

    # --- 1. Loss Curve Plot ---
    train_loss = history['train_loss']
    val_loss = history['test_loss']
    epochs = range(1, len(train_loss) + 1)

    axes[0].plot(epochs, train_loss, label='Training Loss', color='blue', lw=1.5, marker='x', ms=4)
    axes[0].plot(epochs, val_loss, label='Validation Loss', color='orange', lw=1.5, marker='x', ms=4)
    axes[0].set_title('Loss Curve (Learning Curve)')
    axes[0].set_xlabel('Epochs')
    axes[0].set_ylabel('Loss')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)

    # --- 2. Accuracy Plot ---
    train_acc = history['train_acc']
    val_acc = history['test_acc']
    axes[1].plot(epochs, train_acc, label='Training Accuracy', color='blue', lw=1.5, marker='x', ms=4)
    axes[1].plot(epochs, val_acc, label='Validation Accuracy', color='orange', lw=1.5, marker='x', ms=4)
    axes[1].set_title('Accuracy Over Epochs')
    axes[1].set_xlabel('Epochs')
    axes[1].set_ylabel('Accuracy')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)

    # --- 3. Confusion Matrix ---
    actual = np.array(y_true).flatten()
    predicted = np.array(y_pred).flatten()
    
    tp = np.sum((actual == 1) & (predicted == 1))
    tn = np.sum((actual == 0) & (predicted == 0))
    fp = np.sum((actual == 0) & (predicted == 1))
    fn = np.sum((actual == 1) & (predicted == 0))
    cm = np.array([[tn, fp], [fn, tp]])
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Stay', 'Churn'], 
            yticklabels=['Stay', 'Churn'],
            ax=axes[2])
    axes[2].set_title('Confusion Matrix: Churn Prediction')
    axes[2].set_ylabel('Actual')
    axes[2].set_xlabel('Predicted')
    # Adjust layout to prevent overlap
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()
    
evaluate_model_performance(history, y_test, y_hat, title='Model 2: 2 Hidden Layers')